In [1]:
# Install flory
!pip install flory

import numpy as np
DTYPE = np.float64
import flory
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib widget
import os
import sys
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go


In [2]:
def floryHuggins(phi:DTYPE, chi:np.array):
    part_1 = np.sum(phi*np.log(phi))
    part_2 = 0

    for i in range(len(phi)):
        for j in range(i+1, len(phi)):
            part_2 += chi[i][j]*phi[i]*phi[j]

    return part_1 + part_2

In [3]:
# Interaction matrix
chis = np.array([[0, 3.0, 3.0], [3.0, 0.0, 3.0], [3.0, 3.0, 0.0]], dtype = DTYPE)

# Global value of phis
phi_global = np.array([0.4, 0.3, 0.3], dtype = DTYPE)

# Equilibrium 3 phase solution
phases = flory.find_coexisting_phases(3, chis, phi_global, progress=False)

print("\n" + f"Equlibrium phase volumes: {phases.volumes}")
print("Equlibrium phase concentrations:\n" + f"{phases.fractions}")

# Take the transpose as the notation is different:
# For me: phi_ij = component i in compartment j
# For flory: phi_ji = component i in compartment j (OR I THINK SO!)

phases.fractions = np.transpose(phases.fractions)


Equlibrium phase volumes: [0.42640114 0.28679782 0.28680105]
Equlibrium phase concentrations:
[[0.81090938 0.09454572 0.09454498]
 [0.09453783 0.81092273 0.09453935]
 [0.09453967 0.09454194 0.81091835]]


In [4]:
# Free energy of the 3 phase system

F_flory = 0
for i in range(len(phases.volumes)):
    F_flory += phases.volumes[i]*floryHuggins(phases.fractions[:, i], chis)

print(F_flory)


-0.12914677532620267


In [5]:
# Merging compartments 2 and 3

# First, calculate the modified compositions in the hybrid compartment (1+2)
eta_mod = phases.volumes[1] + phases.volumes[2]
phi_1mod = (phases.volumes[1]*phases.fractions[0, 1] + phases.volumes[2]*phases.fractions[0, 2])/eta_mod
phi_2mod = (phases.volumes[1]*phases.fractions[1, 1] + phases.volumes[2]*phases.fractions[1, 2])/eta_mod
phi_3mod = 1 - phi_1mod - phi_2mod


phi_in_kmod = [phi_1mod, phi_2mod, phi_3mod]
phi_in_k1 = [phases.fractions[0, 0], phases.fractions[1, 0], phases.fractions[2, 0]]

# Calculate the Free Energy of this modified system
# i.e. Free Energy of the proposal

F_mod = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[0]*floryHuggins(phi_in_k1, chis)
print(F_mod)

-0.09455153889336784


In [6]:
phi_1mod, phi_2mod

(0.09453875262290926, 0.4527303173814663)

In [7]:
# Effectively 2 degrees of freedom in the system
step_size = DTYPE(0.001)
n_points = 1000

phi_1mod_perturbed = np.linspace(phi_1mod-n_points*step_size, phi_1mod+n_points*step_size, 2*n_points+1)
phi_2mod_perturbed = np.linspace(phi_2mod-n_points*step_size, phi_2mod+n_points*step_size, 2*n_points+1)

accepted_phi_1mod_perturbed = []
accepted_phi_2mod_perturbed = []
accepted_Fs_mod = []

In [8]:
phi_1mod, phi_2mod

(0.09453875262290926, 0.4527303173814663)

In [9]:
np.min(phi_1mod_perturbed), np.max(phi_1mod_perturbed)

(-0.9054612473770908, 1.0945387526229093)

In [10]:
ctr = 0

for phi_1m in phi_1mod_perturbed:
    for phi_2m in phi_2mod_perturbed:
        if phi_1m<0 or phi_2m<0 or phi_1m>1 or phi_2m>1:
            continue
        
        phi_3m = 1 - phi_1m - phi_2m
        if phi_3m<0 or phi_3m>1:
            continue
            
        phi_11mod = (phi_global[0] - phi_1m*(eta_mod))/(phases.volumes[0])
        phi_21mod = (phi_global[1] - phi_2m*(eta_mod))/(phases.volumes[0])
       
        if phi_11mod<0 or phi_11mod>1 or phi_21mod<0 or phi_21mod>1:
            continue
        
        phi_31mod = 1 - phi_11mod - phi_21mod
        if phi_31mod<0 or phi_31mod>1:
            continue

        ctr += 1

        phi_in_kmod = [phi_1m, phi_2m, phi_3m]
        phi_in_k1 = [phi_11mod, phi_21mod, phi_31mod]
        
        F = 0
        F = eta_mod*floryHuggins(phi_in_kmod, chis) + phases.volumes[0]*floryHuggins(phi_in_k1, chis)

        accepted_phi_1mod_perturbed.append(phi_1m)
        accepted_phi_2mod_perturbed.append(phi_2m)
        accepted_Fs_mod.append(F)

In [11]:
ctr/len(phi_1mod_perturbed)**2

0.05661711872699333

In [12]:
df = pd.DataFrame()
df["phi_1mod"] = accepted_phi_1mod_perturbed
df["phi_2mod"] = accepted_phi_2mod_perturbed
df["F"] = accepted_Fs_mod

df

,phi_1mod,phi_2mod,F
0,0.000539,0.47673,0.004651
1,0.000539,0.47773,0.002402
2,0.000539,0.47873,0.000669
3,0.000539,0.47973,-0.000796
4,0.000539,0.48073,-0.002077
...,...,...,...
226690,0.696539,0.29873,0.006033
226691,0.696539,0.29973,0.007769
226692,0.696539,0.30073,0.009659
226693,0.696539,0.30173,0.011760


In [ ]:
min_F_row = df[df['F'] == df['F'].min()]
min_F_row

In [ ]:
min_F_row["phi_1mod"].values[0], min_F_row["phi_2mod"].values[0]

In [ ]:
phi_1mod, phi_2mod

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=pd.Series(df["phi_1mod"].values.ravel()),
    y=pd.Series(df["phi_2mod"].values.ravel()),
    z=pd.Series(df["F"].values.ravel()),
    mode='markers',
    marker=dict(
        size=2,
        color=pd.Series(df["F"].values.ravel()),  # Color by F values
        colorscale='Viridis',
        opacity = 0.1,
        colorbar=dict(
            title='F',  # Colorbar title
            titleside='right',
            titlefont=dict(size=14),
            thickness=20,
            len=0.75,
            x=1.1  # Position adjustment
        ),
        cmin=df["F"].min(),  # Set color scale range
        cmax=df["F"].max()
    ),
    showlegend=False  # Add this to each trace

))

fig.add_trace(go.Scatter3d(
    x=pd.Series(phi_1mod.ravel()),
    y=pd.Series(phi_2mod.ravel()),
    z=pd.Series(F_mod.ravel()),
    mode='markers',
    marker=dict(
        size=5,
        color='orange',
        symbol='circle'
    ),
    showlegend=False  # Add this to each trace
))

fig.update_layout(
    title={
    'text': "C2+C3",  # Title text
    'y':0.95,  # Position from top (0-1)
    'x':0.35,   # Center the title
    'xanchor': 'center',
    'yanchor': 'top',
    'font': dict(
        size=20,  # Title font size
        color='black'  # Title color
    )
    },

    width=1200,  # Width in pixels
    height=800,   # Height in pixels
    autosize=False,  # Disable autosizing
    scene=dict(
        xaxis_title='phi_1mod',
        yaxis_title='phi_2mod',
        zaxis_title='F_sys',
        aspectmode='manual',  # Preserve aspect ratio
        aspectratio=dict(x=1, y=1, z=0.7)  # Adjust these values as needed
    ),
    margin=dict(l=0, r=500, b=0, t=30)  # Increased right margin for colorbar
)

fig.show()